# Modèles de mixture (MM) et Algorithme EM (Expectation - Maximization)
---

## 1. Formulation mathématique
### 1.1 L'algorithme EM

On se donne en ensemble $(X_i)_{i \in \mathbb{N}^*} \in X$ variables observées et $(Z_i)_{i \in \mathbb{N}^*} \in Z$ variables cachées. On cherche à optimiser un ensemble de paramètres $\theta$ à partir du *likelyhood* (on cherche le maximum de vraisemblance). En considérant des données observées $(x_i)_{i \in \mathbb{N}^*}$ i.i.d., le *likelyhood* s'exprime sous la forme :

$$
\begin{align}
\mathcal{L}(\theta) &= p(X \mid \theta)                   \\ 
                    &= p(x_1, \dots, x_N \mid \theta)     \\
                    &= \prod_{i=1}^N p(x_n \mid \theta)     \\
\mathcal{L}(\theta) &= \prod_{i=1}^N p(X = x_n \mid \theta)
\end{align}
$$

Ici, on y ajoute les variables cachées $Z_i$ pas nécéssairement i.i.d., i.e.

$$
\begin{align}
\mathcal{L}(\theta) &= p(X \mid \theta)                                    \\
                    &= \sum_{Z} p(X,Z \mid \theta)                          \\
                    &= \sum_{i=1}^N p(x_1, \dots, x_N, z_i \mid \theta)   \\
                    &= \prod_{i=1}^N \sum_{i=1}^N p(x_n, z_i \mid \theta) \\
\mathcal{L}(\theta) &= \prod_{i=1}^N \sum_{i=1}^N p(X = x_n, Z = z_i \mid \theta)
\end{align}
$$

Pour simplifier le calcul du produit, on passe au $\log$, ce qui donne :

$$
\log \mathcal{L}(\theta) = \log p(X \mid \theta) = \log \prod_{i=1}^N \sum_{i=1}^N p(X = x_n, Z = z_i \mid \theta) = \sum_{n=1}^N \log( \sum_{i=1}^N p(X = x_n, Z = z_i \mid \theta) )
$$

La forme "$\log \sum$" étant difficile à optimiser, on introduit donc une densité de probabilité (d.d.p. ou p.d.f) $q(Z)$ qui approxime le *posterior* $p(Z \mid X, \theta)$ tel que :

$$
p(Z \mid X, \theta) \approx q(Z)
$$

Puis on introduit l'ELBO (evidence lower bound) définit par la moyenne de la log-probabilité jointe $p(X, Z \mid \theta)$ retranchée à l'entropie de $q$. L'ELBO est une borne inférieure de $p(X \mid \theta)$ qui permet sous certaines conditions de calculer le *likelyhood*. Autrement dit :

$$
\text(ELBO)(q,\theta) =
\underbrace{
\mathbb{E}_{q(Z)}[ \log p(X,Z \mid \theta) ]
}_{\text{moyenne de la log-proba jointe}}
-(
\underbrace{
\mathbb{E}_{q(Z)}[ \log q(Z) ]
}_{\text{Entropie de } q(Z)}
) 
\quad \text{qui induit} \quad
\log p(X| \theta) = \log \mathcal{L}(q,Z) + KL(q(Z) || p(Z \mid X, \theta)) \geq \mathcal{L}(q,Z)
$$

Avec $KL(\bullet || \bullet)$ la divergence de Kullback-Liebler, une mesure de la distance entre deux p.d.f. SOient $p$ et $q$ deux p;d.f. La divergence de Kullback-Liebler s'exprime sous la forme :

$$
KL(q||p) = \sum_{z = 1}^Z q(z_i) \log \frac{q(z_i)}{p(z_i)}
$$

En particulier si $q(Z) = p(Z | X, \theta)$, KL = 0 et par conséquent $p(X| \theta) = \mathcal{L}(q,Z)$. L'algorithme EM cherche donc à toruver un $q(Z)$ qui approxime $p(Z | X, \theta)$ de telle sorte à maximiser une approximation du *likelyhood*.

### 1.2 Etapes de l'algorithme

* Etape E (Expectation)

On met à jour la distribution sur les variables cachées :

$$
q(Z) \leftarrow p(Z \mid X, \theta^{(t)})
$$

* Etape M (Maximization)

On met à jour les paramètres en maximisant l’espérance de la log-vraisemblance (moyenne du *Likelyhood*) :

$$
\theta^{(t+1)} \leftarrow \arg\max_{\theta} \; \mathbb{E}_{q(Z)} \left[ \log p(X,Z \mid \theta) \right]
$$

### 1.3 Les modèles de mixture

Un modèle de mixture est une combinaison linéaire de distributions usuelles.

$$

$$

---
#### Retour sur les densités de probabilité usuelles




---

## 2. Un exemple en Python
---
#### 2.1 Import des librairies

In [2]:
import numpy as np

Dans cet exemple, on génère un mélange de deux gaussiennes. On cherche à savoir laquelle des deux gaussiennes a généré le point $x_n$ : c'est la variable cachée. On note donc $Z \in \{1;2\}$ le numéro de la gaussienne.

In [5]:


# -----------------------------
# Données simulées
# -----------------------------
np.random.seed(0)

N = 300
X1 = np.random.normal(loc=-2, scale=1, size=N//2)
X2 = np.random.normal(loc=3, scale=1.5, size=N//2)
X = np.concatenate([X1, X2])

# -----------------------------
# Initialisation des paramètres
# -----------------------------
K = 2  # nombre de clusters

mu = np.random.choice(X, K)          # moyennes initiales
sigma = np.ones(K)                   # écarts-types initiaux
pi = np.ones(K) / K                  # poids initiaux

# -----------------------------
# Fonction densité gaussienne
# -----------------------------
def gaussian(x, mu, sigma):
    return (1 / (np.sqrt(2*np.pi) * sigma)) * np.exp(-0.5 * ((x - mu) / sigma)**2)

# -----------------------------
# Algorithme EM
# -----------------------------
max_iter = 100

for t in range(max_iter):

    # ===== E-step : calcul des responsabilités =====
    gamma = np.zeros((N, K))

    for k in range(K):
        gamma[:, k] = pi[k] * gaussian(X, mu[k], sigma[k])

    # normalisation pour que somme_k gamma[n,k] = 1
    gamma = gamma / gamma.sum(axis=1, keepdims=True)

    # ===== M-step : mise à jour des paramètres =====
    Nk = gamma.sum(axis=0)

    for k in range(K):
        mu[k] = (gamma[:, k] @ X) / Nk[k]
        sigma[k] = np.sqrt((gamma[:, k] @ ((X - mu[k])**2)) / Nk[k])
        pi[k] = Nk[k] / N

    # log-likelihood
    likelihood = np.sum(np.log(
        sum(pi[k] * gaussian(X, mu[k], sigma[k]) for k in range(K))
    ))

    print(f"Iter {t+1:02d} | mu={mu} | sigma={sigma} | pi={pi} | logL={likelihood:.3f}")


Iter 01 | mu=[ 3.69863696 -1.00488823] | sigma=[1.05003648 1.75234107] | pi=[0.32206412 0.67793588] | logL=-698.165
Iter 02 | mu=[ 3.54090695 -1.08363663] | sigma=[1.14290341 1.74116683] | pi=[0.34459294 0.65540706] | logL=-693.851
Iter 03 | mu=[ 3.44340743 -1.19161637] | sigma=[1.18439797 1.66794625] | pi=[0.36711027 0.63288973] | logL=-690.721
Iter 04 | mu=[ 3.36721355 -1.30441853] | sigma=[1.21086476 1.57313656] | pi=[0.38837968 0.61162032] | logL=-687.529
Iter 05 | mu=[ 3.2981072  -1.41304813] | sigma=[1.2341565 1.4728592] | pi=[0.40817941 0.59182059] | logL=-684.326
Iter 06 | mu=[ 3.2326917  -1.51155464] | sigma=[1.258832   1.37683135] | pi=[0.42609573 0.57390427] | logL=-681.372
Iter 07 | mu=[ 3.17144746 -1.59567037] | sigma=[1.28599208 1.29245368] | pi=[0.44169641 0.55830359] | logL=-678.936
Iter 08 | mu=[ 3.11589961 -1.66351686] | sigma=[1.31476169 1.224205  ] | pi=[0.45475537 0.54524463] | logL=-677.153
Iter 09 | mu=[ 3.06732117 -1.71588196] | sigma=[1.34333442 1.17261591] | p